<a href="https://colab.research.google.com/github/pcsilcan/pcd/blob/master/20202/pcd_20202_0902_conway.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conway's problem

In [ ]:
!sudo apt install golang-go

In [ ]:
%%writefile 1.go
package main

import (
    "fmt"
)

const (
    MAX = 9
    K   = 4
)

func compress(inC, pipe chan rune) {
    n := 0
    previous := <-inC
    for c := range inC {
        if c == previous && n < MAX-1 {
            n++
        } else {
            if n > 0 {
                pipe<- rune(n+1 + 48)
                n = 0
            }
            pipe<- previous
            previous = c
        }
    }
    close(pipe)
}

func output(pipe, outC chan rune) {
    m := 0
    for c := range pipe {
        outC<- c
        m++
        if m == K {
            outC <- '\n'
            m = 0
        }
    }
    close(outC)
}

func main() {
    inC     := make(chan rune)
    pipe    := make(chan rune)
    outC    := make(chan rune)

    go compress(inC, pipe)
    go output(pipe, outC)

    go func() {
        str := "aasbdsssdbsbdjjjfffskskkjsjhsshss."
        for _, c := range str {
            inC<- c
        }
        close(inC)
    }()

    for c := range outC {
        fmt.Printf("%c", c)
    }
    fmt.Println()
}

Overwriting 1.go


In [ ]:
!go run 1.go

2asb
d3sd
bsbd
3j3f
sks2
kjsj
h2sh
2s


In [ ]:
package main

import (
	"fmt"
)

var end chan bool

func zero(n int, west chan float64) {
	for i := 0; i < n; i++ {
		west <- 0.0
	}
	close(west)
}

func source(row []float64, south chan float64) {
	for _, element := range row {
		south <- element
	}
	close(south)
}

func sink(north chan float64) {
	for range north {
	}
}

func result(c [][]float64, i int, east chan float64) {
	j := 0
	for element := range east {
		c[i][j] = element
		j++
	}
	end <- true
}

func multiplier(firstElement float64, north, east, south, west chan float64) {
	for secondElement := range north {
		sum := <-east
		sum = sum + firstElement*secondElement
		south <- secondElement
		west <- sum
	}
	close(south)
	close(west)
}

func main() {
	a := [][]float64{{1,  2,  3,  4},
                     {5,  6,  7,  8},
                     {9, 10, 11, 12}}
	b := [][]float64{{1, 0, 2}, {0, 1, 2}, {1, 0, 0}}
	c := [][]float64{{0, 0, 0}, {0, 0, 0}, {0, 0, 0}}
	end = make(chan bool)
	nra := len(a)
	nca := len(a[0])
	ns := make([][]chan float64, nra+1) // canales norte sur, matrix de 4x3
	for i := range ns {
		ns[i] = make([]chan float64, nca)
		for j := range ns[i] {
			ns[i][j] = make(chan float64)
		}
	}
	ew := make([][]chan float64, nra) // canales easte oeste, matrix de 3x4
	for i := range ew {
		ew[i] = make([]chan float64, nca+1)
		for j := range ew[i] {
			ew[i][j] = make(chan float64)
		}
	}
	for i := 0; i < nra; i++ {
		go zero(nra, ew[i][nca])
		go result(c, i, ew[i][0])
	}
	for i := 0; i < nca; i++ {
		go source(b[i], ns[0][i])
		go sink(ns[nra][i])
	}
	for i := 0; i < nra; i++ {
		for j := 0; j < nca; j++ {
			go multiplier(a[i][j],
				ns[i][j], ew[i][j+1],
				ns[i+1][j], ew[i][j])
		}
	}
	for i := 0; i < nra; i++ {
		<-end
	}
	fmt.Println(c)
}

In [ ]:
%%writefile 3.go
package main

import (
	"fmt"
)

var end chan bool

func zero(n int, west chan float64) {
	for i := 0; i < n; i++ {
		west <- 0.0
	}
	close(west)
}

func source(row []float64, south chan float64) {
	for _, element := range row {
		south <- element
	}
	close(south)
}

func sink(north chan float64) {
	for range north {
	}
}

func result(c [][]float64, i int, east chan float64) {
	j := 0
	for element := range east {
		c[i][j] = element
		j++
	}
	end <- true
}

func multiplier(firstElement float64, north, east, south, west chan float64) {
	for secondElement := range north {
		sum := <-east
		sum = sum + firstElement*secondElement
		south <- secondElement
		west <- sum
	}
	close(south)
	close(west)
}

func main() {
	a := [][]float64{{ 1,  2,  3,  4},
                     { 5,  6,  7,  8}}
	b := [][]float64{{ 1,  2},
                     { 5,  6},
                     { 9, 10},
                     {13, 14}}
	c := [][]float64{{0, 0},
	                 {0, 0}}
	end = make(chan bool)
	nra := len(a)
	nca := len(a[0])
	ns := make([][]chan float64, nra+1) // canales norte sur, matrix de 4x3
	for i := range ns {
		ns[i] = make([]chan float64, nca)
		for j := range ns[i] {
			ns[i][j] = make(chan float64)
		}
	}
	ew := make([][]chan float64, nra) // canales easte oeste, matrix de 3x4
	for i := range ew {
		ew[i] = make([]chan float64, nca+1)
		for j := range ew[i] {
			ew[i][j] = make(chan float64)
		}
	}
	for i := 0; i < nra; i++ {
		go zero(nra, ew[i][nca])
		go result(c, i, ew[i][0])
	}
	for i := 0; i < nca; i++ {
		go source(b[i], ns[0][i])
		go sink(ns[nra][i])
	}
	for i := 0; i < nra; i++ {
		for j := 0; j < nca; j++ {
			go multiplier(a[i][j],
				ns[i][j], ew[i][j+1],
				ns[i+1][j], ew[i][j])
		}
	}
	for i := 0; i < nra; i++ {
		<-end
	}
	fmt.Println(c)
}

Overwriting 3.go


In [ ]:
!go run 3.go

[[90 100] [202 228]]


In [ ]:
import numpy as np
a = np.array([[ 1,  2,  3,  4],
              [ 5,  6,  7,  8]])
b = np.array([[ 1,  2],
              [ 5,  6],
              [ 9, 10],
              [13, 14]])
a.dot(b)

array([[ 90, 100],
       [202, 228]])